# Main

In [1]:
# Imports
import os
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics 

from package.processtransformer import constants
from package.processtransformer.models import transformer
from package.processtransformer.data.loader import LogsDataLoader
from package.processtransformer.data.processor import LogsDataProcessor

In [2]:
data_dir = "./datasets/"
if not os.path.exists(data_dir): 
  os.mkdir(data_dir)

In [3]:
dataset_name = "sepsis"
data_processor = LogsDataProcessor(name=dataset_name, filepath="sepsis.xes",  
                                    columns = ["Case ID", "Activity", "Complete Timestamp"], #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', datetime_format="%Y-%m-%d %H:%M:%S.%f", pool = 4)
data_processor.process_logs(task=constants.Task.NEXT_ACTIVITY, sort_temporally= False)

c:\Users\vince\anaconda3\envs\MasterThesis\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
# Load data
data_loader = LogsDataLoader(name = dataset_name)

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.NEXT_ACTIVITY)

# Prepare training examples for next activity prediction task
train_token_x, train_token_y = data_loader.prepare_data_next_activity(train_df, 
    x_word_dict, y_word_dict, max_case_length)

In [5]:
learning_rate = 0.001
batch_size = 12
epochs = 10

In [6]:
# Create and train a transformer model
transformer_model = transformer.get_next_activity_model(
    max_case_length=max_case_length, 
    vocab_size=vocab_size,
    output_dim=num_output)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

transformer_model.fit(train_token_x, train_token_y, 
    epochs=epochs, batch_size=batch_size)

Epoch 1/10
1121/1121 [==============================] - 25s 18ms/step - loss: 0.7965 - sparse_categorical_accuracy: 0.7435
Epoch 2/10
1121/1121 [==============================] - 22s 20ms/step - loss: 0.6575 - sparse_categorical_accuracy: 0.7917
Epoch 3/10
1121/1121 [==============================] - 21s 19ms/step - loss: 0.6439 - sparse_categorical_accuracy: 0.7895
Epoch 4/10
1121/1121 [==============================] - 20s 18ms/step - loss: 0.6409 - sparse_categorical_accuracy: 0.7915
Epoch 5/10
1121/1121 [==============================] - 21s 19ms/step - loss: 0.6389 - sparse_categorical_accuracy: 0.7909
Epoch 6/10
1121/1121 [==============================] - 24s 21ms/step - loss: 0.6367 - sparse_categorical_accuracy: 0.7923
Epoch 7/10
1121/1121 [==============================] - 24s 21ms/step - loss: 0.6337 - sparse_categorical_accuracy: 0.7918
Epoch 8/10
1121/1121 [==============================] - 28s 25ms/step - loss: 0.6329 - sparse_categorical_accuracy: 0.7940
Epoch 9/10
1121/

In [7]:
# Evaluate over all the prefixes (k) and save the results
k, accuracies,fscores, precisions, recalls = [],[],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_token_y = data_loader.prepare_data_next_activity(test_data_subset, 
            x_word_dict, y_word_dict, max_case_length)   
        y_pred = np.argmax(transformer_model.predict(test_token_x), axis=1)
        accuracy = metrics.accuracy_score(test_token_y, y_pred)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            test_token_y, y_pred, average="weighted")
        k.append(i)
        accuracies.append(accuracy)
        fscores.append(fscore)
        precisions.append(precision)
        recalls.append(recall)

k.append(i + 1)
accuracies.append(np.mean(accuracy))
fscores.append(np.mean(fscores))
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))

29/29 [==============================] - 4s 21ms/step


c:\Users\vince\anaconda3\envs\MasterThesis\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


29/29 [==============================] - 1s 23ms/step


c:\Users\vince\anaconda3\envs\MasterThesis\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


28/28 [==============================] - 1s 17ms/step


...

1/1 [==============================] - 0s 232ms/step


c:\Users\vince\anaconda3\envs\MasterThesis\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1/1 [==============================] - 0s 192ms/step


c:\Users\vince\anaconda3\envs\MasterThesis\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1/1 [==============================] - 0s 263ms/step


In [8]:
print('Average accuracy across all prefixes:', np.mean(accuracies))
print('Average f-score across all prefixes:', np.mean(fscores))
print('Average precision across all prefixes:', np.mean(precisions))
print('Average recall across all prefixes:', np.mean(recalls))

Average accuracy across all prefixes: 0.8457020247184004
Average f-score across all prefixes: 0.7903511426927857
Average precision across all prefixes: 0.7954760709421075
Average recall across all prefixes: 0.8328438601116004
